In [ ]:
pip install joblib

In [ ]:
pip install tdqm

In [ ]:
import numpy as np
import pandas as pd
import joblib
from tqdm import tqdm

# TensorFlow 및 TPU 설정
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
#데이터 호출 시리즈
Data_list = ['/kaggle/input/data-set-24-08-09/SOL_Data_1m_Micro_Indicator3.csv',
             '/kaggle/input/data-set-24-08-09/SOL_Data_3m_Indicator3.csv',
             '/kaggle/input/data-set-24-08-09/SOL_Data_5m_Indicator3.csv',
             '/kaggle/input/data-set-24-08-09/SOL_Data_15m_Indicator3.csv',
             '/kaggle/input/data-set-24-08-09/SOL_Data_30m_Indicator3.csv']

Data_save = ['/kaggle/working/SOL_Data_1m_Micro_INDICATOR3_TCN_v4.pth',
             '/kaggle/working/SOL_Data_3m_Micro_INDICATOR3_TCN_v4.pth',
             '/kaggle/working/SOL_Data_5m_Micro_INDICATOR3_TCN_v4.pth',
             '/kaggle/working/SOL_Data_15m_Micro_INDICATOR3_TCN_v4.pth',
             '/kaggle/working/SOL_Data_30m_Micro_INDICATOR3_TCN_v4.pth']

Data_Test_list = ['/kaggle/input/data-set-24-08-09/SOL_Data_Test_1m_Indicator3.csv',
                  '/kaggle/input/data-set-24-08-09/SOL_Data_Test_3m_Indicator3.csv',
                  '/kaggle/input/data-set-24-08-09/SOL_Data_Test_5m_Indicator3.csv',
                  '/kaggle/input/data-set-24-08-09/SOL_Data_Test_15m_Indicator3.csv',
                  '/kaggle/input/data-set-24-08-09/SOL_Data_Test_30m_Indicator3.csv']

Target_List = [0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2.1]

mention_List = ["1m", "3m", "5m", "15m", "30m"]

In [ ]:
#데이터 호출 시리즈
Data_list = ['/kaggle/input/data-set-24-08-09/SOL_Data_1m_Micro_Indicator3.csv']

Data_save = ['/kaggle/working/SOL_Data_1m_INDICATOR3_TCN_v4.pth']

Data_Test_list = ['/kaggle/input/data-set-24-08-09/SOL_Data_Test_1m_Indicator3.csv']

Target_List = [0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2.1]

mention_List = ["1m"]

In [ ]:
#데이터 호출 시리즈
Data_list = ['/kaggle/input/data-set-24-08-09/SOL_Data_3m_Indicator3.csv']

Data_save = ['/kaggle/working/SOL_Data_3m_INDICATOR3_TCN_v4.pth']

Data_Test_list = ['/kaggle/input/data-set-24-08-09/SOL_Data_Test_3m_Indicator3.csv']

Target_List = [0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2.1]

mention_List = ["3m"]

In [ ]:
#함수 시리즈
def drop_unnamed_column(df):
    # 'Unnamed: 0' 열이 존재하는지 확인
    if 'Unnamed: 0' in df.columns:
        # 존재한다면 해당 열 삭제
        df = df.drop(columns=['Unnamed: 0'])
    return df

# 시간 관련 열 변환 함수
def convert_time_features(data):
    # open_time 열이 datetime 형식이 아닌 경우 변환
    if not np.issubdtype(data['open_time'].dtype, np.datetime64):
        data['open_time'] = pd.to_datetime(data['open_time'])

    # time 열을 분 단위로 변환
    data['time'] = data['open_time'].dt.hour * 60 + data['open_time'].dt.minute

    # 사용하지 않을 열 제외
    data = data.drop(columns=['open_time'])

    return data

# 데이터 전처리 함수
def preprocess_data(data, point):
    # 목표 변수 생성
    data['target'] = (data['max_return_60min'] >= point).astype(int)

    # 특성과 목표 변수 분리
    X = data.drop(columns=['max_return_60min', 'min_return_60min', 'target'])
    y = data['target']

    # 무한대 값을 NaN으로 대체
    X.replace([np.inf, -np.inf], np.nan, inplace=True)

    # NaN 값을 평균으로 대체
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)

    # 데이터 정규화
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X_imputed)

    return X_scaled, y

# 시계열 데이터 형태로 변환 함수
def create_sequences(data, target, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length + 1):
        seq = data[i:i + sequence_length]
        label = target[i + sequence_length - 1]
        sequences.append(seq)
        targets.append(label)
    return np.array(sequences), np.array(targets)

# 예측 데이터를 시퀀스 형태로 변환
def create_sequences_for_prediction(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length + 1):
        seq = data[i:i + sequence_length]
        sequences.append(seq)
    return np.array(sequences)

# TCN 모델 정의
class TCNModel(nn.Module):
    def __init__(self, input_size, num_channels, kernel_size=2, dropout=0.2):
        super(TCNModel, self).__init__()
        self.tcn = nn.Conv1d(input_size, num_channels, kernel_size, padding=kernel_size//2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_channels, 1)

    def forward(self, x):
        x = x.transpose(1, 2)  # (batch_size, seq_len, input_size) -> (batch_size, input_size, seq_len)
        y1 = self.tcn(x)
        y1 = self.relu(y1)
        y1 = self.dropout(y1)
        y1 = y1[:, :, -1]
        o = self.fc(y1)
        return o

def calculate_max_min_returns(df):
    window_size = 120

    # 'open_time' 열이 데이터프레임에 있는지 확인
    if 'open_time' not in df.columns:
        raise KeyError("'open_time' 열이 데이터프레임에 포함되어 있어야 합니다.")

    # 'open_time' 열을 datetime으로 변환
    df['open_time'] = pd.to_datetime(df['open_time'])

    # 인덱스 중복 확인 및 제거
    df = df.drop_duplicates(subset='open_time', keep='first').copy()

    # 인덱스를 'open_time'으로 설정
    df.set_index('open_time', inplace=True)

    # 현재 가격
    current_price = df['close']

    # 60분 윈도우를 적용하여 최대 및 최소 가격 계산
    rolling_max = df['high'].rolling(window=window_size, min_periods=1).max().shift(-window_size)
    rolling_min = df['low'].rolling(window=window_size, min_periods=1).min().shift(-window_size)

    # 현재 행의 close와 60분 윈도우 내의 최대 high와 최소 low를 비교하여 수익률 계산
    df['max_return_60min'] = ((rolling_max - current_price) / current_price) * 100
    df['min_return_60min'] = ((rolling_min - current_price) / current_price) * 100

    # 결측값을 적절히 처리 (예: 마지막 몇 행)
    df['max_return_60min'].fillna(0, inplace=True)
    df['min_return_60min'].fillna(0, inplace=True)

    # 인덱스 리셋
    df.reset_index(inplace=True)

    return df

In [ ]:
for idx in range(5) :

  print(f"---------START({mention_List[idx]})------------")

  # 테스트 데이터 호출
  data_test = pd.read_csv(Data_Test_list[idx])

  # 최대 상승률과 최대 하락률 계산
  data_test = calculate_max_min_returns(data_test)

  #Unnamed: 0 열 제거
  data_test = drop_unnamed_column(data_test)

  # 테스트 데이터 시간 관련 열 변환
  data_test = convert_time_features(data_test)

  # 테스트 데이터 사용하지 않을 열 제외
  data_test_predict = data_test.drop(columns=['max_return_60min', 'min_return_60min'])

  # 테스트 데이터 무한대 값을 NaN으로 대체
  data_test_predict.replace([np.inf, -np.inf], np.nan, inplace=True)

  # 테스트 데이터 NaN 값을 평균으로 대체
  imputer = SimpleImputer(strategy='mean')
  data_test_predict_imputed = imputer.fit_transform(data_test_predict)  # 같은 imputer 사용

  # 테스트 데이터 데이터 정규화
  scaler = MinMaxScaler()
  data_test_predict_scaled = scaler.fit_transform(data_test_predict_imputed)  # 같은 scaler 사용

  # 데이터 호출
  data = pd.read_csv(Data_list[idx])

  # 최대 상승률과 최대 하락률 계산
  data = calculate_max_min_returns(data)

  # 시간 관련 열 변환
  data = convert_time_features(data)

  # 시퀀스 길이 설정
  sequence_length = 60

  # 예측용 시퀀스 데이터 생성
  X_test_seq = create_sequences_for_prediction(data_test_predict_scaled, sequence_length)

  #목표 수익
  percent_point = Target_List[idx]

  for idx2 in range(8) :
    #목표 수익
    percent_point = Target_List[idx2]

    # 데이터 전처리
    X_scaled, y = preprocess_data(data, percent_point)

    # 데이터 길이 체크
    if len(X_scaled) < sequence_length:
        raise ValueError(f"데이터 길이({len(X_scaled)})가 시퀀스 길이({sequence_length})보다 짧습니다.")

    # 시퀀스 데이터 생성
    y_array = y.values  # pandas Series를 numpy array로 변환
    X_seq, y_seq = create_sequences(X_scaled, y_array, sequence_length)

    # 학습 데이터와 검증 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

    # 데이터를 텐서로 변환
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

    # 데이터 로더 생성
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # 모델 설정
    input_size = X_train.shape[2]
    num_channels = 64
    model = TCNModel(input_size, num_channels)

    # 손실 함수 및 옵티마이저 설정
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # 조기 종료 설정
    patience = 5
    best_loss = float('inf')
    patience_counter = 0

    # 학습 및 검증 손실을 저장할 리스트
    train_losses = []
    val_losses = []

    # 학습
    num_epochs = 5  # 최대 에포크 수
    for epoch in range(num_epochs):
        # 학습 단계
        model.train()
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # 검증 단계
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                output = model(X_batch)
                loss = criterion(output, y_batch)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(test_loader)
        val_losses.append(avg_val_loss)

        # 조기 종료 조건 체크
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("조기 종료 조건 충족. 학습을 중지합니다.")
            break

    # 모델 평가
    model.eval()
    with torch.no_grad():
      y_true = []
      y_pred = []
      for X_batch, y_batch in test_loader:
        output = model(X_batch)
        y_true.extend(y_batch.tolist())
        y_pred.extend(torch.sigmoid(output).squeeze().tolist())

    # 이진 분류 결과를 위한 평가 지표 계산
    y_pred = np.array(y_pred) > 0.5
    y_true = y_test_tensor.numpy()

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')

    # 모델 상태 저장
    torch.save(model.state_dict(), Data_save[idx])

    # 슬라이딩 윈도우로 데이터 범위 추출
    num_rows = data_test_predict_scaled.shape[0]

    # 시퀀스 데이터를 텐서로 변환
    X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32)

    # 모델을 GPU로 이동 (가능한 경우)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    X_test_tensor = X_test_tensor.to(device)

    #window_size = [43175, 86350, 129600, 172700, 216000, 259200, 302400] #3,4,5,6,7
    window_size = [43175, 86350, 129600, 172700, 216000, 259200, 302400]

    #prediction 결과 저장
    results = []

    # 배치 크기 설정
    batch_size = 63

    #
    data_test_tmp = data_test

    for idx_window in range(4) :

      #prediction 결과 저장
      results = []

      for end in tqdm(range(num_rows, window_size[idx_window] - 1, -batch_size)):
          start = max(end - batch_size + 1, 0)

          # 해당 범위에 대한 시퀀스 텐서 추출
          X_test_tensor_tmp = X_test_tensor[start:end]

          # 예측 수행
          model.eval()
          with torch.no_grad():
              predictions = torch.sigmoid(model(X_test_tensor_tmp)).squeeze().cpu().numpy()

          # 예측 결과를 이진 분류로 변환 (0 또는 1)
          predictions = (predictions > 0.5).astype(int)

          # 예측 결과의 마지막 값을 추가
          if len(predictions.shape) > 0:
              results.append(predictions[-1])
          else:
              results.append(predictions)

      # 원래 순서대로 변경
      results = results[::-1]  # 원래 순서대로 변경

      data_test_tmp['Predictions'] = np.nan
      data_test_tmp.loc[data_test_tmp.index[-len(results):], 'Predictions'] = results
      data_test_tmp = data_test_tmp.dropna(subset=['Predictions'])

      # 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
      count_max_return_ge_1_prediction_0 = len(data_test_tmp[(data_test_tmp['max_return_60min'] >= 1.1) & (data_test_tmp['Predictions'] == 1)])

      # 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
      count_max_return_lt_1_prediction_1 = len(data_test_tmp[(data_test_tmp['max_return_60min'] < 1.1) & (data_test_tmp['Predictions'] == 0)])

      #
      tmp1 = count_max_return_ge_1_prediction_0/len(data_test_tmp)*100
      tmp2 = count_max_return_lt_1_prediction_1/len(data_test_tmp)*100

      #
      print(f"[max_return_60min/{mention_List[idx]}/{percent_point}이상/1/{window_size[idx_window]}] : {tmp1}")
      print(f"[max_return_60min/{mention_List[idx]}/{percent_point}미만/0/{window_size[idx_window]}] : {tmp2}")
      print(f"[확률/{mention_List[idx]}/{percent_point}/{window_size[idx_window]}] : {tmp1 + tmp2}")


  print(f"---------END({mention_List[idx]})------------")